# 1. Usando exemplos de Tweets já baixados
Esses tweets foram baixados utilizando o Flume. Caso você queria baixar novos tweets, seguir para o tópico 2.

## Copiando os arquivos locais para o HDFS

In [ ]:
!hdfs dfs -copyFromLocal $(pwd)/examples/* /user/lucasbatista/

## Trend Topics com MapReduce

In [ ]:
!python mrjob-hashtag-rank.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar hdfs:///user/lucasbatista/*

### Apagando os vértices e arestas do grafo 

In [ ]:
from py2neo import Graph
graph = Graph("bolt://127.0.0.1:7687")
graph.delete_all()

print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.nodes)))

## Criando o grafo de Hashtags e Tweets usando MRJob

In [ ]:
!python twitterMRJob.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar hdfs:///user/lucasbatista/*

## Visualizando o grafo

In [ ]:
import sys
sys.path.insert(0, 'resources/python/')

from graphdrawer.visgraph import draw

options = {"User": "id", "Hashtag": "hashtag"}
draw(graph, options, physics=True)

# 2. Criando uma App no Twitter

## Acessar o endereço abaixo e criar uma App: https://apps.twitter.com/

Criar login, senha e logar

Criar uma nova App clicando em Create New App

Definir os detalhes da aplicação: nome, descrição, website, etc

**No menu "Keys and Tokens" gerar as chaves da App para usar na configuração do Flume e substituir no arquivo twitterAgent.conf abaixo.**

# Configurando o agent, source, channel e sink

**Agent**: Apenas um agente chamado *TwitterAgent*

**Source**: Twitter

**Channel**: Memória

**Sink**: Registra os dados no HDFS

In [ ]:
!cat twitterAgent.conf

## Nota: 
Usando as configurações descritas no site do Flume - **TwitterAgent.sources.Twitter.type = org.apache.flume.source.twitter.TwitterSource** - na configuração do source, o arquivo é gerado com caracteres ilegíveis. Assim, iremos utilizar **TwitterAgent.sources.Twitter.type = com.cloudera.flume.source.TwitterSource** e temos que copiar o arquivo flume-sources-1.0-SNAPSHOT.jar para a pasta do Flume para o seu correto funcionamento.

In [ ]:
!cp resources/flume-sources-1.0-SNAPSHOT.jar ~/resources/local/flume-${FLUME_VERSION}/lib

## Apagando os tweets antigos

In [ ]:
!hdfs dfs -rm /user/lucasbatista/*

# Abrir um terminal e iniciar o FlumeAgent

Espere um tempo para o download de alguns tweets
``` bash
flume-ng agent --conf conf --conf-file twitterAgent.conf --name TwitterAgent -Dflume.looger=INFO,console
``` 

# TrendTopics com MapReduce
### Criando um rank de hashtags

In [ ]:
!python mrjob-hashtag-rank.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar hdfs:///user/lucasbatista/*

# Criando grafo de Hashtags e Usuários usando MRJob

In [ ]:
from py2neo import Graph
graph = Graph("bolt://127.0.0.1:7687")
graph.delete_all()

print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.nodes)))

In [ ]:
!python twitterMRJob.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar hdfs:///user/lucasbatista/*

# Visualizando o grafo

In [ ]:
import sys
sys.path.insert(0, 'resources/python/')

from graphdrawer.visgraph import draw

options = {"User": "id", "Hashtag": "hashtag"}
draw(graph, options, physics=True)